In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
# set common ordered ranks for use across taxonomies
tax_ranks = [ # superkingdom
        'realm', 'subrealm', 'kingdom',
        'subkingdom', 'phylum', 'subphylum', 'class', 'subclass', 'order',
        'suborder', 'family', 'subfamily', 'genus', 'subgenus', 'species'
    ]

plot_ranks_spp = [ # superkingdom
        'realm', 'kingdom','phylum', 'class', 'order',
        'family', 'genus', 'species'
    ]
plot_ranks_family = [ # superkingdom
        'realm', 'kingdom','phylum', 'class', 'order',
        'family', #'genus', 'species'
    ]

# Expanding the ICTV VMR database

## 1. ICTV - VMR Representative genomes + their taxonomy

In [3]:
# ICTV VMR lineages
vmr_lineages = 'from-farm/vmr_MSL38_v1.taxonomy.csv'
vmr = pd.read_csv(vmr_lineages)
# force use GCA version of identifiers 
vmr['GCA_ident'] = vmr['ident'].str.replace('GCF', 'GCA')
# now truncate the version so we match across assembly versions
vmr['GCA_ident'] = vmr['GCA_ident'].str.split('.', expand=True)[0]

# homogenize VMR lineage format by removing 'Viruses' and replacing ',' with ';'
vmr['lineage'] = vmr['lineage'].str.split('Viruses,', expand=True)[1].str.rsplit(',', n=1, expand=True)[0]
vmr['lineage'] = vmr['lineage'].str.replace(',', ';')
vmr.head()

,ident,genbank_ident,refseq_ident,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,...,subfamily,genus,subgenus,species,name,lineage,exemplar_or_additional,gene_accs,protein_accs,GCA_ident
0,GCF_003950175.1,MK064563,NC_048128,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Alphalipothrixvirus,NaN,Alphalipothrixvirus SBFV2,Sulfolobales Beppu filamentous virus 2,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_048128.1,"YP_009817896.1,YP_009817897.1,YP_009817898.1,Y...",GCA_003950175
1,GCF_003441495.1,MH447526,NC_048037,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Alphalipothrixvirus,NaN,Alphalipothrixvirus SFV1,Sulfolobus filamentous virus 1,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_048037.1,"YP_009808111.1,YP_009808112.1,YP_009808113.1,Y...",GCA_003441495
2,GCF_000878935.1,AM087120,NC_010155,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Betalipothrixvirus,NaN,Acidianus filamentous virus 3,Acidianus filamentous virus 3,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_010155.1,"YP_001604343.1,YP_001604344.1,YP_001604345.1,Y...",GCA_000878935
3,GCF_000871385.1,AM087121,NC_010152,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Betalipothrixvirus,NaN,Acidianus filamentous virus 6,Acidianus filamentous virus 6,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_010152.1,"YP_001604159.1,YP_001604160.1,YP_001604161.1,Y...",GCA_000871385
4,GCF_000872245.1,AM087122,NC_010153,Viruses,Adnaviria,NaN,Zilligvirae,NaN,Taleaviricota,NaN,...,NaN,Betalipothrixvirus,NaN,Acidianus filamentous virus 7,Acidianus filamentous virus 7,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...,E,NC_010153.1,"YP_001604225.1,YP_001604226.1,YP_001604227.1,Y...",GCA_000872245


In [4]:
vmr['lineage'].tolist()[1]


'Adnaviria;;Zilligvirae;;Taleaviricota;;Tokiviricetes;;Ligamenvirales;;Lipothrixviridae;;Alphalipothrixvirus;;Alphalipothrixvirus SFV1'

In [5]:
len(vmr['lineage'].tolist()[1].split(';'))


15

In [6]:
vmr['lineage'].nunique()

11052

In [7]:
vmr['realm'].unique()

array(['Adnaviria', 'Duplodnaviria', 'Monodnaviria', 'Riboviria',
       'Ribozyviria', 'Varidnaviria', nan], dtype=object)

In [8]:
vmr[vmr['realm'].isna()]

,ident,genbank_ident,refseq_ident,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,...,subfamily,genus,subgenus,species,name,lineage,exemplar_or_additional,gene_accs,protein_accs,GCA_ident
12039,GCF_000843745.1,AP006270,NC_004690,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Alphabaculovirus,NaN,Alphabaculovirus adhonmai,Adoxophyes honmai nucleopolyhedrovirus,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...,E,NC_004690.1,"NP_818648.1,NP_818649.1,NP_818650.1,NP_818651....",GCA_000843745
12040,GCF_000883155.1,EU839994,NC_011345,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Alphabaculovirus,NaN,Alphabaculovirus agipsilonis,Agrotis ipsilon multiple nucleopolyhedrovirus,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...,E,NC_011345.1,"YP_002268031.1,YP_002268032.1,YP_002268033.1,Y...",GCA_000883155
12041,GCF_000868985.1,DQ123841,NC_007921,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Alphabaculovirus,NaN,Alphabaculovirus agsegetum,Agrotis segetum nucleopolyhedrovirus A,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...,E,NC_007921.1,"YP_529671.1,YP_529672.1,YP_529673.1,YP_529674....",GCA_000868985
12042,GCF_000928115.1,KM102981,NC_025960,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Alphabaculovirus,NaN,Alphabaculovirus alteragsegetum,Agrotis segetum nucleopolyhedrovirus B,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...,E,NC_025960.1,"YP_009112562.1,YP_009112563.1,YP_009112564.1,Y...",GCA_000928115
12043,GCF_000857025.1,AY327402,NC_005137,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Alphabaculovirus,NaN,Alphabaculovirus alterchofumiferanae,Choristoneura fumiferana DEF multiple nucleopo...,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...,E,NC_005137.2,"NP_932609.1,NP_932610.1,NP_932611.1,NP_932612....",GCA_000857025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12625,GCF_002830465.1,KT099179,NC_043203,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Deltasatellite,NaN,Sweet potato leaf curl deltasatellite 3,sweet potato leaf curl deltasatellite 3,;;;;;;;;;;Tolecusatellitidae;;Deltasatellite;;...,E,NC_043203.1,NaN,GCA_002830465
12626,GCF_000843845.1,U74627,NC_002743,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Deltasatellite,NaN,Tomato leaf curl deltasatellite,tomato leaf curl deltasatellite,;;;;;;;;;;Tolecusatellitidae;;Deltasatellite;;...,E,NC_002743.1,NaN,GCA_000843845
12627,GCF_002830485.1,JN819495,NC_043204,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Deltasatellite,NaN,Tomato yellow leaf distortion deltasatellite 1,tomato yellow leaf distortion deltasatellite 1,;;;;;;;;;;Tolecusatellitidae;;Deltasatellite;;...,E,NC_043204.1,NaN,GCA_002830485
12628,GCF_002830505.1,KU232893,NC_043205,Viruses,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Deltasatellite,NaN,Tomato yellow leaf distortion deltasatellite 2,tomato yellow leaf distortion deltasatellite 2,;;;;;;;;;;Tolecusatellitidae;;Deltasatellite;;...,E,NC_043205.1,NaN,GCA_002830505


### Taxonomic distribution VMR

In [9]:
plot_ranks_spp

['realm', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

In [10]:
vmr_plot = vmr.fillna('NA') # appease plotly
vm_sp_count = vmr.groupby(plot_ranks_spp).size().reset_index(name='count')
vm_sp_count.head()

,realm,kingdom,phylum,class,order,family,genus,species,count
0,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Alphalipothrixvirus,Alphalipothrixvirus SBFV2,1
1,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Alphalipothrixvirus,Alphalipothrixvirus SFV1,1
2,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 3,1
3,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 6,1
4,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 7,1


In [11]:
vmr_lincount = vm_sp_count

In [12]:
vmr_lincount['count'].unique()

array([ 1,  2,  5, 13,  4,  6, 12, 10, 11,  3,  8,  7, 21, 20,  9, 17, 15,
       14, 24, 25, 63, 27, 80, 30, 55, 18, 35])

In [13]:
vmr_lincount.head()

,realm,kingdom,phylum,class,order,family,genus,species,count
0,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Alphalipothrixvirus,Alphalipothrixvirus SBFV2,1
1,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Alphalipothrixvirus,Alphalipothrixvirus SFV1,1
2,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 3,1
3,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 6,1
4,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 7,1


In [14]:
fig = px.sunburst(vm_sp_count, path=plot_ranks_spp, values='count',
                  #color="phylum",
                  color_discrete_sequence=px.colors.qualitative.Antique)
#fig.update_layout(uniformtext=dict(minsize=8, mode='hide'),margin=dict(l=20, r=20, t=20, b=20))
fig.write_html('vmr-species-taxonomy.sunburst.html', auto_open=True)
#fig.show(renderer='iframe')
#fig.show("png")

In [15]:
vmr_lincount[vmr_lincount['kingdom'] == 'NA'].head()

,realm,kingdom,phylum,class,order,family,genus,species,count


In [16]:
vm_fam_count = vmr.groupby(plot_ranks_family).size().reset_index(name='count')
vm_fam_count.head()

,realm,kingdom,phylum,class,order,family,count
0,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,10
1,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Rudiviridae,17
2,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Ungulaviridae,1
3,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Maximonvirales,Ahmunviridae,1
4,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Primavirales,Tristromaviridae,3


In [17]:
fig = px.sunburst(vm_fam_count, path=plot_ranks_family, values='count',
                  #color="phylum",
                  color_discrete_sequence=px.colors.qualitative.Antique)
fig.write_html('vmr-taxonomy.family.sunburst.html', auto_open=True)
#fig.show(renderer='iframe')
#fig.show("png")

In [18]:
vmr.shape

(12630, 25)

## GenBank NCBI lineages for 62k genbank genomes

In [19]:
# genbank 60k genomes: lineages
gb_ncbi_lineages = 'from-farm/genbank-2023.05.viral.ncbi-lineages.csv'
gb = pd.read_csv(gb_ncbi_lineages)
# force use GCA version of identifiers 
gb['GCA_ident'] = gb['ident'].str.replace('GCF', 'GCA')
gb['GCA_ident'] = gb['GCA_ident'].str.split('.', expand=True)[0]
# rename gb 'clade' --> 'realm' so they match
gb.rename(columns={"clade":'realm'}, inplace=True)

# build 'lineage' column
# Combine the columns with ';' as separator, filling NaNs with empty string
gb['lineage'] = gb[tax_ranks].fillna('').apply(lambda row: ';'.join(row.values.astype(str)), axis=1)

In [20]:
gb.shape

(62306, 20)

In [21]:
gb.head()

,ident,taxid,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,class,subclass,order,suborder,family,subfamily,genus,subgenus,species,GCA_ident,lineage
0,GCA_000839185.1,10243,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,NaN,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_000839185,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...
1,GCA_003971385.1,10243,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,NaN,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_003971385,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...
2,GCA_003971405.1,10243,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,NaN,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_003971405,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...
3,GCA_004025355.1,10243,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,NaN,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_004025355,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...
4,GCA_004025395.1,10243,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,NaN,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_004025395,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...


In [22]:
len(gb['lineage'].tolist()[1].split(';'))

15

In [23]:
gb['lineage'].nunique()

26608

### Plot taxonomic distribution of NCBI lineages for GenBank Genomes

In [24]:
gb_plot = gb.fillna('NA') # appease plotly
gb_sp_count = gb_plot.groupby(plot_ranks_spp).size().reset_index(name='count')

In [25]:
gb_sp_count.head()

,realm,kingdom,phylum,class,order,family,genus,species,count
0,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Alphalipothrixvirus,Alphalipothrixvirus SBFV2,1
1,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Alphalipothrixvirus,Alphalipothrixvirus SFV1,2
2,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 3,1
3,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 6,1
4,Adnaviria,Zilligvirae,Taleaviricota,Tokiviricetes,Ligamenvirales,Lipothrixviridae,Betalipothrixvirus,Acidianus filamentous virus 7,1


In [26]:
gb_sp_count['count'].unique()

array([   1,    2,    3,    4,    6,   22,   28,  338,   27,   85,  586,
          5,   12,   66,   19,   20,   37,  107,   33,   50,   11,   13,
          7,   55,   53,    8,   14,   87,   42,   25,   39,   46,   18,
         62,   16,   10,   77,    9,   17,   29,  389,   76,  624,   59,
       1895,  196,   58,   23,   36,   38,   34,   15,   26,   68,   31,
         75,  362,  108,  169, 1762,  159,  155,   24,   80,  114,   69,
        419,  113,   64, 2423,  249,  336,  126,   54,  401, 1359,  601,
         30,  869,   47,  387,  142,  151,  367,  608,  213,  171, 2514,
         40,   97,  197,  130,   71,   49,   44,   95,   63,  499,   21,
        210,   74,  118,   92, 5343,  148,  637,  233,  239])

In [27]:
fig = px.sunburst(gb_sp_count, path=plot_ranks_spp, values='count',
                  color_discrete_sequence=px.colors.qualitative.Dark2)
#fig.update_layout(uniformtext=dict(minsize=8, mode='hide'),margin=dict(l=20, r=20, t=20, b=20))
fig.write_html('gb-taxonomy.species.sunburst.html', auto_open=True)
#fig.show(renderer='iframe')
#fig.show("png")

In [28]:
gb_fam_count = gb_plot.groupby(plot_ranks_family).size().reset_index(name='count')
fig = px.sunburst(gb_fam_count, path=plot_ranks_family, values='count',
                  color_discrete_sequence=px.colors.qualitative.Dark2)
#fig.update_layout(uniformtext=dict(minsize=8, mode='hide'),margin=dict(l=20, r=20, t=20, b=20))
fig.write_html('gb-taxonomy.family.sunburst.html', auto_open=True)
#fig.show(renderer='iframe')
#fig.show("png")

In [29]:
gb_plot[gb_plot['realm'] =='NA']

,ident,taxid,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,class,subclass,order,suborder,family,subfamily,genus,subgenus,species,GCA_ident,lineage
7358,GCA_000846205.1,10449,Viruses,NA,NA,NA,NA,NA,NA,Naldaviricetes,NA,Lefavirales,NA,Baculoviridae,NA,Alphabaculovirus,NA,Alphabaculovirus lydisparis,GCA_000846205,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...
7359,GCA_000529295.1,10454,Viruses,NA,NA,NA,NA,NA,NA,Naldaviricetes,NA,Lefavirales,NA,Baculoviridae,NA,Alphabaculovirus,NA,Spodoptera exigua multiple nucleopolyhedrovirus,GCA_000529295,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...
7360,GCA_000529585.1,10454,Viruses,NA,NA,NA,NA,NA,NA,Naldaviricetes,NA,Lefavirales,NA,Baculoviridae,NA,Alphabaculovirus,NA,Spodoptera exigua multiple nucleopolyhedrovirus,GCA_000529585,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...
7361,GCA_000530085.1,10454,Viruses,NA,NA,NA,NA,NA,NA,Naldaviricetes,NA,Lefavirales,NA,Baculoviridae,NA,Alphabaculovirus,NA,Spodoptera exigua multiple nucleopolyhedrovirus,GCA_000530085,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...
7362,GCA_000530135.1,10454,Viruses,NA,NA,NA,NA,NA,NA,Naldaviricetes,NA,Lefavirales,NA,Baculoviridae,NA,Alphabaculovirus,NA,Spodoptera exigua multiple nucleopolyhedrovirus,GCA_000530135,;;;;;;Naldaviricetes;;Lefavirales;;Baculovirid...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62260,GCA_029910915.1,3038357,Viruses,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,Pseudomonas phage Y7,GCA_029910915,;;;;;;;;;;;;;;Pseudomonas phage Y7
62261,GCA_029910925.1,3038358,Viruses,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,Pseudomonas phage Y8,GCA_029910925,;;;;;;;;;;;;;;Pseudomonas phage Y8
62286,GCA_029910935.1,3038982,Viruses,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,Staphylococcus phage phiST9-A,GCA_029910935,;;;;;;;;;;;;;;Staphylococcus phage phiST9-A
62287,GCA_029910945.1,3038983,Viruses,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,Staphylococcus phage phiST9-B,GCA_029910945,;;;;;;;;;;;;;;Staphylococcus phage phiST9-B


## GenBank-VMR Lineage Comparisons

### How many NCBI lineages match ICTV-VMR?

In [30]:
vmr_lins = vmr[['GCA_ident', 'lineage']].rename(columns={'lineage': 'lineage_vmr'})
gb_lins = gb[['GCA_ident', 'lineage']].rename(columns={'lineage': 'lineage_gb'})

In [31]:
merged_df = pd.merge(vmr_lins, gb_lins, on='GCA_ident', how='inner')

In [32]:
merged_df['identical_lineage'] = merged_df['lineage_vmr'] == merged_df['lineage_gb']

In [33]:
merged_df.GCA_ident.nunique()

12500

In [34]:
identical_count = merged_df['identical_lineage'].sum()
print(f"Number of identical lineages: {identical_count}")

Number of identical lineages: 10866


In [35]:
diff_lins = merged_df[merged_df['identical_lineage'] == False]
diff_lins.tail()

,GCA_ident,lineage_vmr,lineage_gb,identical_lineage
12475,GCA_000889475,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;T...,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;T...,False
12477,GCA_000872085,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;T...,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;T...,False
12478,GCA_002988005,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;T...,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;T...,False
12486,GCA_000844325,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;Z...,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;Z...,False
12487,GCA_002988015,;;;;;;;;;;Tolecusatellitidae;;Deltasatellite;;...,;;;;;;;;;;Tolecusatellitidae;;Betasatellite;;C...,False


In [36]:
diff_lins.shape

(1634, 4)

### Can we identify which ICTV MSL the GenBank lineages are from?

#### Read in ICTV Master Species Lists (2015-2022) and Changelog (2021->2022 only)

- these will necessarily have overlapping lineages, as anything that did not change between versions will be identical from one file to the next.

In [37]:
# all MSL
msl_csv = 'from-farm/all-msl.csv'
# MSL: changes for v38
msl_changes = 'from-farm/msl-changes.csv'

In [38]:
msl = pd.read_csv(msl_csv)
msl.tail()

,msl_name,lineage
104295,MSL30,;;;;;;;;Unassigned;;Virgaviridae;;Tobamovirus;...
104296,MSL30,;;;;;;;;Unassigned;;Virgaviridae;;Tobamovirus;...
104297,MSL30,;;;;;;;;Unassigned;;Virgaviridae;;Tobravirus;;...
104298,MSL30,;;;;;;;;Unassigned;;Virgaviridae;;Tobravirus;;...
104299,MSL30,;;;;;;;;Unassigned;;Virgaviridae;;Tobravirus;;...


In [39]:
msl['msl_name'].unique()

array(['MSL38.v3', 'MSL38.v2', 'MSL38.v1', 'MSL37.v3', 'MSL37.v2',
       'MSL37.v1', 'MSL36', 'MSL35', 'MSL34', 'MSL33', 'MSL32', 'MSL31',
       'MSL30'], dtype=object)

In [40]:
msl[msl['msl_name'] == 'MSL38.v3'].shape

(11273, 2)

#### Create a mapping of lineage: msl name

In [41]:
# Group by 'lineage' and aggregate 'msl_name' into a list; create dict
lineage_to_msl = msl.groupby('lineage')['msl_name'].apply(list)
l2msl = lineage_to_msl.to_dict()

In [42]:
# now lets use this dict to annotate the GB genomes with the version of MSL that the annotation was from, if possible
merged_df['msl_name'] = merged_df['lineage_gb'].map(lineage_to_msl_mapping)
merged_df.tail()

NameError: name 'lineage_to_msl_mapping' is not defined

In [ ]:
diff_lins = merged_df[merged_df['identical_lineage'] == False]
diff_lins.head()

In [ ]:
# Count the number of NaNs in the 'msl_name' column
nan_count = merged_df['msl_name'].isna().sum()

# Print the count
print(f"Number of NaNs in the 'msl_name' column: {nan_count}")

In [ ]:
# how many times are the GenBank lineages in merged_df reused in the full GenBank Assembly datasets?

gb[]


In [ ]:
def print_lins(df, GCA_ident_value, gb_col='lineage_gb', vmr_col='lineage_vmr'):
    """
    print lins for a given GCA_ident in a DataFrame.
    """
    if GCA_ident_value in df['GCA_ident'].values:
        lineage_gb_value = df.loc[df['GCA_ident'] == GCA_ident_value, gb_col].iloc[0]
        lineage_vmr_value = df.loc[df['GCA_ident'] == GCA_ident_value, vmr_col].iloc[0]
    else:
        lineage_gb_value = lineage_vmr_value = 'Not Found'
    print(f"{GCA_ident_value}")
    print(f"GB: {lineage_gb_value}")
    print(f"VMR: {lineage_vmr_value}")

In [ ]:
print_lins(merged_df,'GCA_002988005.1')

In [ ]:
print_lins(merged_df, 'GCA_002988015.1')

In [ ]:
print_lins(merged_df, 'GCA_027939225.1')

In this case, "cacatuid alphaherpesvirus 2" actually got moved to the 'name' portion :/

Ok, can't find all of these in the VMR, unfortunately. 
BUT, we can just create a mapping between these lineages, since we know they are equivalent

We know there are only 26k unique lineages in GB dataset. 
Let's figure out how many of these we can directly translate using the VMR or the MSL files

In [ ]:
#create genbank: vmr lineage map
gb_to_vmr = merged_df.set_index('lineage_gb')['lineage_vmr'].to_dict()

In [ ]:
gb['vmr_lineage'] = gb['lineage'].map(gb_to_vmr)
gb['vmr_lineage'].head()

In [ ]:
# Count the number of NaNs in the 'vmr_lineage' column
nan_count = gb['vmr_lineage'].isna().sum()

# Print the count
print(f"Number of NaNs in the 'vmr_lineage' column: {nan_count}")

In [ ]:
gb_nodirect = gb[gb['vmr_lineage'].isna()]
gb_nodirect.shape

In [ ]:
gb_nodirect['lineage'].nunique()

In [ ]:
gb_nodirect.head()

In [ ]:
# now let's see if we can identify any lineages from the MSL
# now lets use this dict to annotate the GB genomes with the version of MSL that the annotation was from, if possible
gb_nodirect['msl_name'] = gb_nodirect['lineage'].map(lineage_to_msl_mapping)
gb_nodirect.head()

In [ ]:
gb_nodirect.tail()

In [ ]:
# Count the number of NaNs in the 'msl_name' column
nan_count = gb_nodirect['msl_name'].isna().sum()

# Print the count
print(f"Number of NaNs in the 'msl_name' column: {nan_count}")

In [ ]:
gb_nolin = gb_nodirect.copy()[gb_nodirect['msl_name'].isna()]
gb_nolin.head()

In [ ]:
# let's look at a couple to see what options we have...
print_lins(gb_nolin, 'GCA_002833685', gb_col='lineage', vmr_col='vmr_lineage')

**It looks like "Invertebrate iridescent virus 1" existed up to MSL33**
MSL33,;;;;;;;;;;Iridoviridae;Betairidovirinae;Iridovirus;;Invertebrate iridescent virus 1
MSL32,;;;;;;;;Unassigned;;Iridoviridae;Betairidovirinae;Iridovirus;;Invertebrate iridescent virus 1
MSL31,;;;;;;;;Unassigned;;Iridoviridae;Betairidovirinae;Iridovirus;;Invertebrate iridescent virus 1
MSL30,;;;;;;;;Unassigned;;Iridoviridae;;Iridovirus;;Invertebrate iridescent virus 1

After that, the Invertebrate iridescent viruses are found as part of two different genera, Iridovirus and Chloriridovirus.

**Here are all the entries in MSL38**
MSL38.v3,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota;;Megaviricetes;;Pimascovirales;;Iridoviridae;Betairidovirinae;Chloriridovirus;;Invertebrate iridescent virus 3
MSL38.v3,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota;;Megaviricetes;;Pimascovirales;;Iridoviridae;Betairidovirinae;Chloriridovirus;;Invertebrate iridescent virus 9
MSL38.v3,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota;;Megaviricetes;;Pimascovirales;;Iridoviridae;Betairidovirinae;Chloriridovirus;;Invertebrate iridescent virus 22
MSL38.v3,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota;;Megaviricetes;;Pimascovirales;;Iridoviridae;Betairidovirinae;Chloriridovirus;;Invertebrate iridescent virus 25
MSL38.v3,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota;;Megaviricetes;;Pimascovirales;;Iridoviridae;Betairidovirinae;Iridovirus;;Invertebrate iridescent virus 6
MSL38.v3,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota;;Megaviricetes;;Pimascovirales;;Iridoviridae;Betairidovirinae;Iridovirus;;Invertebrate iridescent virus 31

#### This example implies that we could look for lineages that match to the genus level, and accept those (they would be non-ICTV ratified species in the same genus)

In [ ]:
# there are only 84 subgenera, but let's keep it in here anyway (shrug)
def get_subgenus_lineage_map(lineage_dict):
    return {";".join(key.split(';')[:-1]): value for key, value in lineage_dict.items()}

In [ ]:
msl_subgenus_lineage_map = get_subgenus_lineage_map(lineage_to_msl_mapping)

In [ ]:
#how many can we match?
gb_nolin["subgenus_lineage"] = gb_nolin['lineage'].str.rsplit(';', n=1, expand=True)[0]

In [ ]:
gb_nolin['subgenus_msl'] = gb_nolin["subgenus_lineage"].map(msl_subgenus_lineage_map)

In [ ]:
gb_nolin.head()

In [ ]:
# Count the number of NaNs in the 'msl_name' column
nan_count = gb_nolin['subgenus_msl'].isna().sum()

# Print the count
print(f"Number of NaNs in the 'msl_name' column: {nan_count}")

In [ ]:
gb_nolin.shape

In [ ]:
23617-16936

In [ ]:
print_lins(gb_nolin, 'GCA_002833685', gb_col = 'subgenus_lineage', vmr_col='subgenus_msl')

In [ ]:
len(gb)

In [ ]:
62306-16936

In [ ]:
45370 - 6681

In [ ]:
gb_remains = gb_nolin.copy()[gb_nolin['subgenus_msl'].isna()]
gb_remains.head()

In [ ]:
gb_remains.shape

In [ ]:
print_lins(gb_remains, 'GCA_001274345', gb_col = 'lineage', vmr_col= 'vmr_lineage')

In MSL38, there are x papillomavirus. All share taxonomy to the Family level (Papillomaviridae), with two subfamilies, Firstpapillomavirinae and Secondpapillomavirinae. The majority are Firstpapillomavirinae

for example: 
Monodnaviria;;Shotokuvirae;;Cossaviricota;;Papovaviricetes;;Zurhausenvirales;;Papillomaviridae;Firstpapillomavirinae;

```
MSL38.v1,Monodnaviria;;Shotokuvirae;;Cossaviricota;;Papovaviricetes;;Zurhausenvirales;;Papillomaviridae;Firstpapillomavirinae;Zetapapillomavirus;;Zetapapillomavirus 1
MSL38.v1,Monodnaviria;;Shotokuvirae;;Cossaviricota;;Papovaviricetes;;Zurhausenvirales;;Papillomaviridae;Secondpapillomavirinae;Alefpapillomavirus;;Alefpapillomavirus 1
```

By looking at the NCBI taxonomy browser (https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10566&mode=info), this lineage has 'unclassified Papillomaviridae; Human papillomavirus types'

subfamilies Firstpapillomavirinae and Secondpapillomavirinae were introduced in 2017, so likely this genome cannot be classified to these subfamilies.

**For this genome, the ICTV-ratified annotation stops at the family level.**

In [ ]:
print_lins(gb_remains, 'GCA_000846325', gb_col = 'lineage', vmr_col= 'vmr_lineage')

Most 'Caudoviricetes' annotations in MSL38 start with:
`MSL38.v1,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Caudoviricetes;;;;;;`

Though some have the following orders, some of which were added with MSL38:
Crassvirales
Juravirales
Kirjokansivirales
Magrovirales
Methanobavirales
Nakonvirales
Thumleimavirales


in prior msl, the order Caudovirales can be found, e.g.:
```
MSL36,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Caudoviricetes;;Caudovirales;;Zobellviridae;;...
MSL36,Duplodnaviria;;Heunggongvirae;;Uroviricota;;Caudoviricetes;;Caudovirales;;Siphoviridae;;...
```

suggesting that this classification was updated for MSL37.

Looking at ICTV taxon history, the order 'Caudovirales' was abolished in MSL37 (https://ictv.global/taxonomy/taxondetails?taxnode_id=202000190&taxon_name=Caudovirales)

In [ ]:
# genus_level matches 
def rank_lineage_map(lineage_dict, rank='species'):
    rank_idx = tax_ranks.index(rank)
    return {";".join(key.split(';')[:rank_idx+1]): value for key, value in lineage_dict.items()}

In [ ]:
genus_lineage_map = rank_lineage_map(lineage_to_msl_mapping, 'genus')
family_lineage_map = rank_lineage_map(lineage_to_msl_mapping, 'family')

In [ ]:
genus_index = tax_ranks.index('genus')
family_index = tax_ranks.index('family')
family_index

In [ ]:
gb_remains["genus_lineage"] = gb_remains['lineage'].str.rsplit(';',n=14-genus_index, expand=True)[0]
gb_remains["family_lineage"] = gb_remains['lineage'].str.rsplit(';',n=14-family_index, expand=True)[0]

In [ ]:
# check
print_lins(gb_remains,'GCF_002833685', gb_col='genus_lineage', vmr_col='family_lineage')

In [ ]:
gb_remains['genus_msl'] = gb_remains["genus_lineage"].map(genus_lineage_map)
gb_remains['family_msl'] = gb_remains["family_lineage"].map(family_lineage_map)

In [ ]:
gb_remains[~gb_remains['genus_msl'].isna()]

In [ ]:
gb_remains[~gb_remains['family_msl'].isna()]

In [ ]:
16394-945

In [ ]:
# check for msl38 specifically
contains_MSL38 = gb_remains['family_msl'].apply(lambda x: any('MSL38' in msl_name for msl_name in x) if isinstance(x, list) else False)
cm = gb_remains[contains_MSL38]

cm.shape

In [ ]:
# check for 37 or 38
# Check for rows where 'MSL37' or 'MSL38' is in the 'msl_name_list' column, handling NaN values
contains_MSL37_or_MSL38 = gb_remains['family_msl'].apply(lambda x: any('MSL37' in msl_name or 'MSL38' in msl_name for msl_name in x) if isinstance(x, list) else False)
cm2 = gb_remains[contains_MSL37_or_MSL38]
cm2.shape

## Ok, back to getting ICTV lineages for GenBank Genomes

In [48]:
vmr_lins.head()

,GCA_ident,lineage_vmr
0,GCA_003950175,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...
1,GCA_003441495,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...
2,GCA_000878935,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...
3,GCA_000871385,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...
4,GCA_000872245,Adnaviria;;Zilligvirae;;Taleaviricota;;Tokivir...


In [49]:
gb_with_ictv_lineages = pd.merge(gb, vmr_lins, on='GCA_ident', how='outer')

In [50]:
gb_with_ictv_lineages.head()

,ident,taxid,superkingdom,realm,subrealm,kingdom,subkingdom,phylum,subphylum,class,...,order,suborder,family,subfamily,genus,subgenus,species,GCA_ident,lineage,lineage_vmr
0,GCA_000839185.1,10243.0,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,...,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_000839185,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...
1,GCA_003971385.1,10243.0,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,...,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_003971385,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...,NaN
2,GCA_003971405.1,10243.0,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,...,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_003971405,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...,NaN
3,GCA_004025355.1,10243.0,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,...,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_004025355,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...,NaN
4,GCA_004025395.1,10243.0,Viruses,Varidnaviria,NaN,Bamfordvirae,NaN,Nucleocytoviricota,NaN,Pokkesviricetes,...,Chitovirales,NaN,Poxviridae,Chordopoxvirinae,Orthopoxvirus,NaN,Cowpox virus,GCA_004025395,Varidnaviria;;Bamfordvirae;;Nucleocytoviricota...,NaN
